In [2]:
import pandas as pd
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt

read files

In [3]:
df_2012 = pd.read_csv("Datasets/cbecs_servers_cleaned_2012_dummy.csv", index_col=0)

df_2018 = pd.read_csv("Datasets/cbecs_servers_cleaned_2018_dummy.csv", index_col=0)

report summary

In [4]:
df_2012.groupby('space type').agg(
            Number_of_DCs=('FINALWT', 'sum'),
            Number_of_Servers=('nweight_s', 'sum'),
            Number_of_Samples=('SERVERN', 'count'))

,Number_of_DCs,Number_of_Servers,Number_of_Samples
space type,,,
large,1.843097e+03,8.404435e+06,26
midsize,2.120316e+04,1.472855e+06,360
small,1.724783e+06,3.704352e+06,3163


In [5]:
df_2018[df_2018.SERVERN > 0].groupby('space type').agg(
            Number_of_DCs=('FINALWT', 'sum'),
            Number_of_Servers=('nweight_s', 'sum'),
            Number_of_Samples=('SERVERN', 'count'))

,Number_of_DCs,Number_of_Servers,Number_of_Samples
space type,,,
large,2373.20,12570990.11,85
midsize,13086.10,1308053.92,272
small,1384988.71,2953531.97,3276


In [6]:
def process_data(df, year):
    # Filter the DataFrame
    filtered_df = df[df['space type'].isin(['small', 'midsize'])]

    # Group and aggregate the data
    if year == 2012:
        result = filtered_df.groupby('space type').agg(
            Number_of_Buildings=('FINALWT', 'sum'),
            Number_of_Servers=('nweight_s', 'sum'),
            #Average_Number_of_Servers=('SERVERN', 'mean'),
            Number_of_CBECS_Records=('space type', 'count')
        )
    else:
        result = filtered_df.groupby('space type').agg(
            Number_of_Buildings=('FINALWT', 'sum'),
            Number_of_Servers=('nweight_s', 'sum'),
            #Average_Number_of_Servers=('SERVERN', 'mean'),
            Number_of_Racks=('nweight_r', 'sum'),
            #Average_Number_of_Racks=('RACKN', 'mean'),
            Number_of_CBECS_Records=('space type', 'count')
        )

    # Normalize the values
    result['Number_of_Buildings'] = result['Number_of_Buildings'] / 1e6
    result['Number_of_Servers'] = result['Number_of_Servers'] / 1e6

    if year != 2012:
        result['Number_of_Racks'] = result['Number_of_Racks'] / 1e6
        result['%_of Racks in CBECS'] = result['Number_of_Racks'] / (result['Number_of_Racks'].sum())
        result['Average_Number_of_Racks'] = result['Number_of_Racks']/result['Number_of_Buildings']

    result['%_of Servers in CBECS'] = result['Number_of_Servers'] / (result['Number_of_Servers'].sum())
    result['Average_Number_of_Servers'] = result['Number_of_Servers']/result['Number_of_Buildings']

    return result



In [7]:
pd.options.display.float_format = lambda x: f'{x:.2f}'

In [8]:
a2018 = process_data(df_2018, 2018)
a2018

,Number_of_Buildings,Number_of_Servers,Number_of_Racks,Number_of_CBECS_Records,%_of Racks in CBECS,Average_Number_of_Racks,%_of Servers in CBECS,Average_Number_of_Servers
space type,,,,,,,,
midsize,0.01,1.31,0.44,272,0.10,33.32,0.31,99.96
small,1.38,2.95,3.94,3276,0.90,2.84,0.69,2.13


In [9]:
a2012 = process_data(df_2012, 2012)
a2012

,Number_of_Buildings,Number_of_Servers,Number_of_CBECS_Records,%_of Servers in CBECS,Average_Number_of_Servers
space type,,,,,
midsize,0.02,1.47,360,0.28,69.46
small,1.72,3.70,3163,0.72,2.15


In [10]:
df_2018.groupby(['space type'])['nweight_s'].sum()

space type
large     12570990.11
midsize    1308053.92
small      2953531.97
Name: nweight_s, dtype: float64

In [11]:
df_2012.groupby(['space type'])['nweight_s'].sum()

space type
large     8404435.15
midsize   1472855.27
small     3704351.58
Name: nweight_s, dtype: float64

Servers by Principle Building Activity

In [12]:
# Server Distribution by Principle Building Activity

mapping = {
    'Inpatient health care': 'Medical',
    'Nursing': 'Medical',
    'Outpatient health care': 'Medical',
    'Laboratory': 'Medical',
    'Public order and safety': 'Public order',
    'Enclosed mall': 'Retail',
    'Retail other than mall': 'Retail',
    'Strip shopping mall': 'Retail',
    'Nonrefrigerated warehouse': 'Warehouse',
    'Refrigerated warehouse': 'Warehouse'
}

servers_by_pba_2012 = (df_2012.groupby(['space type','building type'])[['nweight_s']].sum()/df_2012.groupby(['space type'])[['nweight_s']].sum()).reset_index()
servers_by_pba_2012['Percentage'] = servers_by_pba_2012['nweight_s'] * 100
servers_by_pba_2012['Year'] = 2012
servers_by_pba_2012['Principle Building Activity'] = servers_by_pba_2012['building type'].apply(lambda x: mapping[x] if x in mapping else x)
servers_by_pba_2012 = servers_by_pba_2012.drop(columns=['nweight_s'])
servers_by_pba_2012 = servers_by_pba_2012[servers_by_pba_2012['space type'].isin({'midsize', 'small'})]

servers_by_pba_2018 = (df_2018.groupby(['space type','building type'])[['nweight_s']].sum()/df_2018.groupby(['space type'])[['nweight_s']].sum()).reset_index()
servers_by_pba_2018['Percentage'] = servers_by_pba_2018['nweight_s'] * 100
servers_by_pba_2018['Year'] = 2018
servers_by_pba_2018['Principle Building Activity'] = servers_by_pba_2018['building type'].apply(lambda x: mapping[x] if x in mapping else x)
servers_by_pba_2018 = servers_by_pba_2018.drop(columns=['nweight_s'])
servers_by_pba_2018 = servers_by_pba_2018[servers_by_pba_2018['space type'].isin({'midsize', 'small'})]

servers_by_pba = pd.concat([servers_by_pba_2012, servers_by_pba_2018], ignore_index=True)

## Write
with pd.ExcelWriter("Datasets/R report.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    servers_by_pba.to_excel(writer, sheet_name='Data Center Servers by Principle Building Activity', index=False)

/opt/miniconda3/envs/env_1/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [13]:
server_count_by_pba_2012 = df_2012.groupby(['space type','building type'])[['nweight_s']].sum().reset_index()
server_count_by_pba_2012['Year'] = 2012
server_count_by_pba_2012['Principle Building Activity'] = server_count_by_pba_2012['building type'].apply(lambda x: mapping[x] if x in mapping else x)
server_count_by_pba_2012 = server_count_by_pba_2012[server_count_by_pba_2012['space type'].isin({'midsize', 'small'})]

server_count_by_pba_2018 = df_2018.groupby(['space type','building type'])[['nweight_s']].sum().reset_index()
server_count_by_pba_2018['Year'] = 2018
server_count_by_pba_2018['Principle Building Activity'] = server_count_by_pba_2018['building type'].apply(lambda x: mapping[x] if x in mapping else x)
server_count_by_pba_2018 = server_count_by_pba_2018[server_count_by_pba_2018['space type'].isin({'midsize', 'small'})]

server_count_by_pba = pd.concat([server_count_by_pba_2012, server_count_by_pba_2018], ignore_index=True)

server_count_by_pba['count'] = server_count_by_pba['nweight_s']
server_count_by_pba = server_count_by_pba.drop(columns=['nweight_s'])

# Write
with pd.ExcelWriter("Datasets/R report.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    server_count_by_pba.to_excel(writer, sheet_name='Data Center Server Counts by Principle Building Activity', index=False)

/opt/miniconda3/envs/env_1/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Usage Distribution of Data Center Cooling Systems in the U.S.

In [14]:
def generate_cooling(df, index=2012):

    # Direct Expansion
    df['Direct Expansion'] = 0
    df.loc[(df['RCAC'] == 1) | (df['PKGCL'] == 1) | (df['ACWNWL'] == 1) , 'Direct Expansion'] = 1
    # Air Chiller
    df['Air Chiller'] = 0
    df.loc[df['CHLAIRCL']==1, 'Air Chiller'] = 1
    # Water Chiller
    df['Water Chiller'] = 0
    df.loc[df['CHLWTRCL']==1, 'Water Chiller'] = 1
    # Absorption Chiller
    df['Absorption Chiller'] = 0
    df.loc[df['CHLABSRP'] == 1, 'Absorption Chiller'] = 1
    # Chiller (Other)
    df['Chiller (Other)'] = 0
    df.loc[(df['HTPMPC'] == 1) | (df['CHWT'] == 1), 'Chiller (Other)'] = 1 
    # Evaporative Cooling
    df['Evaporative Cooling'] = 0
    df.loc[df['EVAPCL']==1, 'Evaporative Cooling'] = 1

    # Air Economizer
    df['Air Economizer'] = 0
    if index == 2012:
        df.loc[df['ECNTYPE']==1, 'Air Economizer'] = 1
    if index == 2018:
        df.loc[df['ECN']==1, 'Air Economizer'] = 1
    
    # Other Cooling
    df['Other'] = 0
    if index == 2012:
        condition = (df['RCAC'] == 1) | (df['PKGCL'] == 1) | (df['ACWNWL'] == 1)| (df['CHLAIRCL']==1) | (df['CHLWTRCL']==1) | (df['CHLABSRP'] == 1) | (df['HTPMPC'] == 1) | (df['CHWT'] == 1) | (df['EVAPCL']==1) | (df['ECNTYPE']==1)
    if index == 2018:
        condition = (df['RCAC'] == 1) | (df['PKGCL'] == 1) | (df['ACWNWL'] == 1)| (df['CHLAIRCL']==1) | (df['CHLWTRCL']==1) | (df['CHLABSRP'] == 1) | (df['HTPMPC'] == 1) | (df['CHWT'] == 1) | (df['EVAPCL']==1) | (df['ECN']==1)
    df.loc[~(condition) & (df['COOL'] == 1),  'Other'] = 1
    
    return df[['space type','CENDIV','Direct Expansion','Air Chiller', 'Water Chiller', 'Absorption Chiller', 'Chiller (Other)', 'Evaporative Cooling', 'Air Economizer', 'Other', 'FINALWT']]


In [15]:
def generate_cooling_metrics(df, masks = [1,1,1,1,1,1,1,1], filter = None, filter_div = None):
    
    if filter != None:
        df = df[df['space type'] == filter]
    if filter_div != None:
        df = df[df['CENDIV'] == filter_div]

    ALL = df.loc[(df['Direct Expansion'] == 1) | (df['Air Chiller']==1) | (df['Water Chiller']==1) | (df['Absorption Chiller']==1) | (df['Chiller (Other)']==1) | (df['Evaporative Cooling']==1) | (df['Air Economizer']==1) | (df['Other']==1), 'FINALWT'].sum()

    coolings = ['Direct Expansion', 'Air Chiller', 'Water Chiller', 'Absorption Chiller', 'Chiller (Other)', 'Evaporative Cooling', 'Air Economizer', 'Other']
    key = tuple([k for m, k in zip(masks, coolings) if m==1])

    a,b,c,d,e,f,g,h = masks
    ans = df.loc[(df['Direct Expansion'] == a) & (df['Air Chiller']==b) & (df['Water Chiller']==c) & (df['Absorption Chiller']==d) & (df['Chiller (Other)']==e) & (df['Evaporative Cooling']==f) & (df['Air Economizer']==g) & (df['Other']==h), 'FINALWT'].sum()
    return key, ans/ALL*100


#generate_cooling_metrics(df=generate_cooling(df_2018, index=2018), masks = [1,1,1,1,1,1,1,1], filter = None, filter_div=1)

In [16]:
n = 8
all_masks = [[(i >> j) & 1 for j in range(n - 1, -1, -1)] for i in range(2**n)]


In [17]:
mp_dc_2018 = {}
mp_dc_midsize_2018 = {}
mp_dc_small_2018 = {}
mp_dc_midsize_2018_div = {}
mp_dc_small_2018_div = {}

for masks in all_masks:
    if masks == [0,0,0,0,0,0,0,0]:
        continue
    key1, ans1 = generate_cooling_metrics(df=generate_cooling(df_2018, 2018), masks = masks, filter = None)
    key2, ans2 = generate_cooling_metrics(df=generate_cooling(df_2018, 2018), masks = masks, filter = 'midsize')
    key3, ans3 = generate_cooling_metrics(df=generate_cooling(df_2018, 2018), masks = masks, filter = 'small')
    
    mp_dc_2018[key1] = ans1
    mp_dc_midsize_2018[key2] = ans2
    mp_dc_small_2018[key3] = ans3

    for div in [5, 7, 8, 4, 9, 1, 3, 6, 2]:
        key, ans = generate_cooling_metrics(df=generate_cooling(df_2018, 2018), masks = masks, filter = 'midsize', filter_div=div)
        mp_dc_midsize_2018_div[(div, key)] = ans

        key_, ans_ = generate_cooling_metrics(df=generate_cooling(df_2018, 2018), masks = masks, filter = 'small', filter_div=div)
        mp_dc_small_2018_div[(div, key_)] = ans_
        


In [18]:
mp_dc_2012 = {}
mp_dc_midsize_2012 = {}
mp_dc_small_2012 = {}
mp_dc_midsize_2012_div = {}
mp_dc_small_2012_div = {}

for masks in all_masks:
    if masks == [0,0,0,0,0,0,0,0]:
        continue
    key1, ans1 = generate_cooling_metrics(df=generate_cooling(df_2012, 2012), masks = masks, filter = None)
    key2, ans2 = generate_cooling_metrics(df=generate_cooling(df_2012, 2012), masks = masks, filter = 'midsize')
    key3, ans3 = generate_cooling_metrics(df=generate_cooling(df_2012, 2012), masks = masks, filter = 'small')
    
    mp_dc_2012[key1] = ans1
    mp_dc_midsize_2012[key2] = ans2
    mp_dc_small_2012[key3] = ans3

    for div in [5, 7, 8, 4, 9, 1, 3, 6, 2]:
        key, ans = generate_cooling_metrics(df=generate_cooling(df_2012, 2012), masks = masks, filter = 'midsize', filter_div=div)
        mp_dc_midsize_2012_div[(div, key)] = ans

        key_, ans_ = generate_cooling_metrics(df=generate_cooling(df_2012, 2012), masks = masks, filter = 'small', filter_div=div)
        mp_dc_small_2012_div[(div, key_)] = ans_

In [19]:
# Small DC

res_s_2012 = pd.DataFrame(list(mp_dc_small_2012.items()), columns=['Cooling System', 'Percentage'])
res_s_2012['Year'] = 2012
res_s_2012['Type'] = 'Small'

res_s_2018 = pd.DataFrame(list(mp_dc_small_2018.items()), columns=['Cooling System', 'Percentage'])
res_s_2018['Year'] = 2018
res_s_2018['Type'] = 'Small'

threshold = 3
constraints1_s = set(res_s_2012[res_s_2012.Percentage >= threshold]['Cooling System'].unique())
constraints2_s = set(res_s_2018[res_s_2018.Percentage >= threshold]['Cooling System'].unique())
constraints_s = constraints1_s.union(constraints2_s)
# constraints_s = {('Direct Expansion',), ('Direct Expansion', 'Air Economizer')}

res_s_2012 = res_s_2012[res_s_2012['Cooling System'].isin(constraints_s)]
res_s_2018 = res_s_2018[res_s_2018['Cooling System'].isin(constraints_s)]

new_row_2012 = pd.DataFrame({
    'Cooling System': 'Other',
    'Percentage': 100 - res_s_2012.Percentage.sum(),
    'Year': 2012,
    'Type': 'Small'
}, index=[0])
res_s_2012 = pd.concat([res_s_2012, new_row_2012], ignore_index=True)

new_row_2018 = pd.DataFrame({
    'Cooling System': 'Other',
    'Percentage': 100 - res_s_2018.Percentage.sum(),
    'Year': 2018,
    'Type': 'Small'
}, index=[0])
res_s_2018 = pd.concat([res_s_2018, new_row_2018], ignore_index=True)

res_s = pd.concat([res_s_2018, res_s_2012], ignore_index=True)

mp_s = {('Direct Expansion',): 'Direct Expansion',
 ('Direct Expansion', 'Air Economizer'): 'Air Economizer (Direct Expansion)',
 'Other': 'Others',
 ('Chiller (Other)',): 'Other Chillers', 
 ('Direct Expansion', 'Chiller (Other)'): 'Direct Expansion + Other Chillers'}

res_s['Cooling System'] = res_s['Cooling System'].apply(lambda x: mp_s[x])

## Write
with pd.ExcelWriter("Datasets/R report.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    res_s.to_excel(writer, sheet_name='Cooling System (Small Data Centers)', index=False)

/opt/miniconda3/envs/env_1/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [20]:
# Midsize DC

res_m_2012 = pd.DataFrame(list(mp_dc_midsize_2012.items()), columns=['Cooling System', 'Percentage'])
res_m_2012['Year'] = 2012
res_m_2012['Type'] = 'Midsize'

res_m_2018 = pd.DataFrame(list(mp_dc_midsize_2018.items()), columns=['Cooling System', 'Percentage'])
res_m_2018['Year'] = 2018
res_m_2018['Type'] = 'Midsize'

threshold = 4
constraints1_m = set(res_m_2012[res_m_2012.Percentage >= threshold]['Cooling System'].unique())
constraints2_m = set(res_m_2018[res_m_2018.Percentage >= threshold]['Cooling System'].unique())
constraints_m = constraints1_m.union(constraints2_m)

res_m_2012 = res_m_2012[res_m_2012['Cooling System'].isin(constraints_m)]
res_m_2018 = res_m_2018[res_m_2018['Cooling System'].isin(constraints_m)]

new_row_2012 = pd.DataFrame({
    'Cooling System': 'Other',
    'Percentage': 100 - res_m_2012.Percentage.sum(),
    'Year': 2012,
    'Type': 'Midsize'
}, index=[0])

res_m_2012 = pd.concat([res_m_2012, new_row_2012], ignore_index=True)

new_row_2018 = pd.DataFrame({
    'Cooling System': 'Other',
    'Percentage': 100 - res_m_2018.Percentage.sum(),
    'Year': 2018,
    'Type': 'Midsize'
}, index=[0])

res_m_2018 = pd.concat([res_m_2018, new_row_2018], ignore_index=True)

res_m = pd.concat([res_m_2018, res_m_2012], ignore_index=True)

mp_m = {('Direct Expansion', 'Chiller (Other)', 'Air Economizer'): 'Air Economizer (Direct Expansion + Other Chillers)', 
      ('Chiller (Other)',): 'Other Chillers', 
      ('Direct Expansion', 'Water Chiller', 'Air Economizer'): 'Air Economizer (Direct Expansion + Water Chiller)', 
      ('Direct Expansion', 'Air Economizer'): 'Air Economizer (Direct Expansion)', 
      ('Chiller (Other)', 'Air Economizer'): 'Air Economizer (Other Chillers)', 
      ('Water Chiller',): 'Water Chiller', 
      ('Water Chiller', 'Air Economizer'): 'Air Economizer (Water Chiller)', 
      ('Direct Expansion',): 'Direct Expansion',
      ('Air Chiller', 'Air Economizer'): 'Air Economizer (Air Chiller)',
      ('Direct Expansion', 'Evaporative Cooling', 'Air Economizer'): 'Air Economizer + Evaporative Cooling (Direct Expansion)',
      'Other': 'Others'
      }

mp_m2 = {'Air Economizer (Direct Expansion + Other Chillers)': 'Air Economizer (Mechanical Cooling)', 
      'Other Chillers': 'Other Chillers', 
      'Air Economizer (Direct Expansion + Water Chiller)': 'Air Economizer (Mechanical Cooling)', 
      'Air Economizer (Direct Expansion)': 'Air Economizer (Mechanical Cooling)', 
      'Air Economizer (Other Chillers)': 'Air Economizer (Mechanical Cooling)', 
      'Water Chiller': 'Water Chiller', 
      'Air Economizer (Water Chiller)': 'Air Economizer (Mechanical Cooling)', 
      'Direct Expansion': 'Direct Expansion',
      'Air Economizer (Air Chiller)': 'Air Economizer (Mechanical Cooling)', 
      'Air Economizer + Evaporative Cooling (Direct Expansion)': 'Air Economizer + Evaporative Cooling (Direct Expansion)',
      'Others': 'Others'
      }

res_m['Cooling System'] = res_m['Cooling System'].apply(lambda x: mp_m[x])
res_m['Cooling System 2'] = res_m['Cooling System'].apply(lambda x: mp_m2[x])

## Write
with pd.ExcelWriter("Datasets/R report.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    res_m.to_excel(writer, sheet_name='Cooling System (Midsize Data Centers)', index=False)

/opt/miniconda3/envs/env_1/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [21]:
# Small DC by Census Div

res_sdiv_2012 = pd.DataFrame(list(mp_dc_small_2012_div.items()), columns=['Div, Cooling System', 'Percentage'])
res_sdiv_2012['Div'] = res_sdiv_2012['Div, Cooling System'].apply(lambda x: x[0])
res_sdiv_2012['Cooling System'] = res_sdiv_2012['Div, Cooling System'].apply(lambda x: x[1])
res_sdiv_2012['Year'] = 2012
res_sdiv_2012['Type'] = 'Small'
res_sdiv_2012 = res_sdiv_2012.drop(columns=['Div, Cooling System'])

res_sdiv_2018 = pd.DataFrame(list(mp_dc_small_2018_div.items()), columns=['Div, Cooling System', 'Percentage'])
res_sdiv_2018['Div'] = res_sdiv_2018['Div, Cooling System'].apply(lambda x: x[0])
res_sdiv_2018['Cooling System'] = res_sdiv_2018['Div, Cooling System'].apply(lambda x: x[1])
res_sdiv_2018['Year'] = 2018
res_sdiv_2018['Type'] = 'Small'
res_sdiv_2018 = res_sdiv_2018.drop(columns=['Div, Cooling System'])

# using previous constraints
res_sdiv_2012 = res_sdiv_2012[res_sdiv_2012['Cooling System'].isin(constraints_s)]
res_sdiv_2018 = res_sdiv_2018[res_sdiv_2018['Cooling System'].isin(constraints_s)]

# Generate others:
others_2012 = pd.DataFrame(100 - res_sdiv_2012.groupby(['Div'])['Percentage'].sum()).reset_index()
others_2012['Cooling System'] = 'Other'
others_2012['Year'] = 2012
others_2012['Type'] = 'Small'

others_2018 = pd.DataFrame(100 - res_sdiv_2018.groupby(['Div'])['Percentage'].sum()).reset_index()
others_2018['Cooling System'] = 'Other'
others_2018['Year'] = 2018
others_2018['Type'] = 'Small'

res_sdiv_2012 = pd.concat([res_sdiv_2012, others_2012], ignore_index=True)
res_sdiv_2018 = pd.concat([res_sdiv_2018, others_2018], ignore_index=True)

res_sdiv = pd.concat([res_sdiv_2012, res_sdiv_2018], ignore_index=True)
res_sdiv['Cooling System'] = res_sdiv['Cooling System'].apply(lambda x: mp_s[x])

divisions = pd.read_csv("Datasets/geo_mapping.csv")[['division census', 'cdiv num']].drop_duplicates().sort_values(by="cdiv num")
mp_div = dict(zip(divisions['cdiv num'], divisions['division census']))
res_sdiv['Census Division'] = res_sdiv['Div'].apply(lambda x: mp_div[x])

## Write
with pd.ExcelWriter("Datasets/R report.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    res_sdiv.to_excel(writer, sheet_name='Cooling System (Small Data Centers by Census Division)', index=False)

/opt/miniconda3/envs/env_1/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [22]:
# Midsize DC by Census Div

res_mdiv_2012 = pd.DataFrame(list(mp_dc_midsize_2012_div.items()), columns=['Div, Cooling System', 'Percentage'])
res_mdiv_2012['Div'] = res_mdiv_2012['Div, Cooling System'].apply(lambda x: x[0])
res_mdiv_2012['Cooling System'] = res_mdiv_2012['Div, Cooling System'].apply(lambda x: x[1])
res_mdiv_2012['Year'] = 2012
res_mdiv_2012['Type'] = 'Midsize'
res_mdiv_2012 = res_mdiv_2012.drop(columns=['Div, Cooling System'])

res_mdiv_2018 = pd.DataFrame(list(mp_dc_midsize_2018_div.items()), columns=['Div, Cooling System', 'Percentage'])
res_mdiv_2018['Div'] = res_mdiv_2018['Div, Cooling System'].apply(lambda x: x[0])
res_mdiv_2018['Cooling System'] = res_mdiv_2018['Div, Cooling System'].apply(lambda x: x[1])
res_mdiv_2018['Year'] = 2018
res_mdiv_2018['Type'] = 'Midsize'
res_mdiv_2018 = res_mdiv_2018.drop(columns=['Div, Cooling System'])

# using previous constraints
res_mdiv_2012 = res_mdiv_2012[res_mdiv_2012['Cooling System'].isin(constraints_m)]
res_mdiv_2018 = res_mdiv_2018[res_mdiv_2018['Cooling System'].isin(constraints_m)]

# Generate others:
others_2012 = pd.DataFrame(100 - res_mdiv_2012.groupby(['Div'])['Percentage'].sum()).reset_index()
others_2012['Cooling System'] = 'Other'
others_2012['Year'] = 2012
others_2012['Type'] = 'Midsize'

others_2018 = pd.DataFrame(100 - res_mdiv_2018.groupby(['Div'])['Percentage'].sum()).reset_index()
others_2018['Cooling System'] = 'Other'
others_2018['Year'] = 2018
others_2018['Type'] = 'Midsize'

res_mdiv_2012 = pd.concat([res_mdiv_2012, others_2012], ignore_index=True)
res_mdiv_2018 = pd.concat([res_mdiv_2018, others_2018], ignore_index=True)

res_mdiv = pd.concat([res_mdiv_2012, res_mdiv_2018], ignore_index=True)
res_mdiv['Cooling System'] = res_mdiv['Cooling System'].apply(lambda x: mp_m[x])
res_mdiv['Cooling System 2'] = res_mdiv['Cooling System'].apply(lambda x: mp_m2[x])

divisions = pd.read_csv("Datasets/geo_mapping.csv")[['division census', 'cdiv num']].drop_duplicates().sort_values(by="cdiv num")
mp_div = dict(zip(divisions['cdiv num'], divisions['division census']))
res_mdiv['Census Division'] = res_mdiv['Div'].apply(lambda x: mp_div[x])

## Write
with pd.ExcelWriter("Datasets/R report.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    res_mdiv.to_excel(writer, sheet_name='Cooling System (Midsize Data Centers by Census Division)', index=False)

/opt/miniconda3/envs/env_1/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Geospatial Distribution of Servers in the U.S.

In [23]:
cdiv_mapping = pd.read_csv("Datasets/geo_mapping.csv")[['cdiv num', 'division census']].drop_duplicates().sort_values(by="cdiv num").set_index('cdiv num').to_dict()['division census']

servers_by_cdiv_2012 = (df_2012.groupby(['space type','CENDIV'])[['nweight_s']].sum()/df_2012.groupby(['space type'])[['nweight_s']].sum()).reset_index()
servers_by_cdiv_2012['Percentage'] = servers_by_cdiv_2012['nweight_s'] * 100
servers_by_cdiv_2012['Year'] = 2012
servers_by_cdiv_2012['Census Division'] = servers_by_cdiv_2012['CENDIV'].map(cdiv_mapping)
servers_by_cdiv_2012 = servers_by_cdiv_2012.drop(columns=['nweight_s', 'CENDIV'])
servers_by_cdiv_2012 = servers_by_cdiv_2012[servers_by_cdiv_2012['space type'].isin({'midsize', 'small'})]

servers_by_cdiv_2018 = (df_2018.groupby(['space type','CENDIV'])[['nweight_s']].sum()/df_2018.groupby(['space type'])[['nweight_s']].sum()).reset_index()
servers_by_cdiv_2018['Percentage'] = servers_by_cdiv_2018['nweight_s'] * 100
servers_by_cdiv_2018['Year'] = 2018
servers_by_cdiv_2018['Census Division'] = servers_by_cdiv_2018['CENDIV'].map(cdiv_mapping)
servers_by_cdiv_2018 = servers_by_cdiv_2018.drop(columns=['nweight_s', 'CENDIV'])
servers_by_cdiv_2018 = servers_by_cdiv_2018[servers_by_cdiv_2018['space type'].isin({'midsize', 'small'})]

servers_by_cdiv = pd.concat([servers_by_cdiv_2012, servers_by_cdiv_2018], ignore_index=True)

## Write
with pd.ExcelWriter("Datasets/R report.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    servers_by_cdiv.to_excel(writer, sheet_name='Server Distribution by Census Division', index=False)

/opt/miniconda3/envs/env_1/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Geospatial Distribution of Servers in the U.S. (Net Changes)

In [24]:
pba_id = pd.read_csv('Datasets/pba_id.csv',header=0,index_col=0)
pba_id_map = {v: k for k, v in pba_id.to_dict()['building type'].items()}

# 2012
a_2012 = df_2012[df_2012['space type'].isin({'midsize', 'small'})].groupby(['space type', 'CENDIV', 'building type'])[['nweight_s']].sum().reset_index()
a_2012['pba']= a_2012['building type'].map(pba_id_map)
a_2012['cdiv num'] = a_2012['CENDIV']
a_2012 = a_2012.drop(columns=['CENDIV','building type'])

b_2012 = pd.read_csv("Datasets/employment_by_zip_and_pba_2012.csv", index_col=0)

b_2012 = b_2012.merge(a_2012, on=['pba', 'cdiv num'], how='left').fillna(0)
b_2012['server'] = b_2012.frac * b_2012['nweight_s']

c_2012 = b_2012.groupby(['ZIP CODE'])[['server']].sum().reset_index()
c_2012['ZIP CODE'] = c_2012['ZIP CODE'].astype(str).str.zfill(5)
# c_2012.head()

In [25]:
print(df_2012[df_2012['space type'].isin({'midsize', 'small'})]['nweight_s'].sum())
print(c_2012.server.sum())

print(df_2012[df_2012['space type'].isin({'small'})]['nweight_s'].sum())
print(df_2012[df_2012['space type'].isin({'midsize'})]['nweight_s'].sum())

5177206.850336911
5177206.850336322
3704351.5839183563
1472855.2664185544


In [26]:
# 2018
a_2018 = df_2018[df_2018['space type'].isin({'midsize', 'small'})].groupby(['space type', 'CENDIV', 'building type'])[['nweight_s']].sum().reset_index()
a_2018['pba']= a_2018['building type'].map(pba_id_map)
a_2018['cdiv num'] = a_2018['CENDIV']
a_2018 = a_2018.drop(columns=['CENDIV','building type'])

b_2018 = pd.read_csv("Datasets/employment_by_zip_and_pba_2018.csv", index_col=0)

b_2018 = b_2018.merge(a_2018, on=['pba', 'cdiv num'], how='left').fillna(0)
b_2018['server'] = b_2018.frac * b_2018['nweight_s']

c_2018 = b_2018.groupby(['ZIP CODE'])[['server']].sum().reset_index()
c_2018['ZIP CODE'] = c_2018['ZIP CODE'].astype(str).str.zfill(5)
# c_2018.head()

In [27]:
print(df_2018[df_2018['space type'].isin({'midsize', 'small'})]['nweight_s'].sum())
print(c_2018.server.sum())
print(df_2018[df_2018['space type'].isin({'small'})]['nweight_s'].sum())
print(df_2018[df_2018['space type'].isin({'midsize'})]['nweight_s'].sum())

4261585.89
4261585.889999504
2953531.9699999997
1308053.92


In [28]:
# Net Changes
merged_c = pd.merge(c_2012, c_2018, on='ZIP CODE', how='outer', suffixes=('_2012', '_2018'))
merged_c['Server Change'] = merged_c['server_2018'] - merged_c['server_2012']

In [29]:
merged_c

,ZIP CODE,server_2012,server_2018,Server Change
0,01001,136.69,111.73,-24.96
1,01002,136.69,111.73,-24.96
2,01003,136.69,111.73,-24.96
3,01005,130.78,108.35,-22.43
4,01007,136.69,111.73,-24.96
...,...,...,...,...
32979,99923,28.44,20.37,-8.07
32980,99925,28.44,20.37,-8.07
32981,99926,28.44,20.37,-8.07
32982,99927,28.44,20.37,-8.07


In [30]:
# mapping zipcode in merged_c to county
d = pd.read_csv("Datasets/zipcode_county.csv", index_col=0)
d['zipcode'] = d['zipcode'].astype(str).str.zfill(5)
d.head()

,zipcode,county,state,latitude,longitude
0,13309,Oneida County,NY,43.47,-75.32
1,34785,Sumter County,FL,28.84,-82.06
2,46960,Pulaski County,IN,41.17,-86.52
3,54552,Price County,WI,45.94,-90.27
4,62035,Madison County,IL,38.97,-90.24


In [31]:
# zipcode map to fips
d2 = pd.read_csv("Datasets/county_mapping.csv")
d2['county_fips'] = d2['county_fips'].astype(str).str.zfill(5)
d2.head()

,county_name,county_fips,state_abbv
0,Mohave County,04015,AZ
1,Flagler County,12035,FL
2,Morton County,20129,KS
3,Marshall County,28093,MS
4,St. Louis city,29510,MO


In [32]:
df = pd.merge(d, d2, left_on="state", right_on="state_abbv", how="outer")
df = df[(df.county == df.county_name)]

In [33]:
d3 = df[['zipcode', 'county_fips']].copy()

In [34]:
d4 = pd.merge(merged_c, d3, left_on="ZIP CODE", right_on="zipcode", how="outer")


In [35]:
from uszipcode import SearchEngine

def get_county_info2(zipcode):
    search = SearchEngine()
    result = search.by_zipcode(zipcode)

    if result:
        return result.county, result.state
    else:
        return None, None

# get_county_info2("01031")

In [36]:
missing = pd.DataFrame(d4[d4['county_fips'].isna()]['ZIP CODE'])
missing[['county', 'state']] = missing['ZIP CODE'].apply(get_county_info2).apply(pd.Series)

df_missing = pd.merge(missing, d2, left_on="state", right_on="state_abbv", how="outer")
df_missing = df_missing[(df_missing.county == df_missing.county_name)]
df_missing

,ZIP CODE,county,state,county_name,county_fips,state_abbv
14,99566,Valdez-Cordova Census Area,AK,Valdez-Cordova Census Area,02261,AK
77,99645,Matanuska-Susitna Borough,AK,Matanuska-Susitna Borough,02170,AK
106,99652,Matanuska-Susitna Borough,AK,Matanuska-Susitna Borough,02170,AK
135,99654,Matanuska-Susitna Borough,AK,Matanuska-Susitna Borough,02170,AK
222,99674,Matanuska-Susitna Borough,AK,Matanuska-Susitna Borough,02170,AK
...,...,...,...,...,...,...
17921,25060,Raleigh County,WV,Raleigh County,54081,WV
17986,25110,Kanawha County,WV,Kanawha County,54039,WV
18012,25211,Clay County,WV,Clay County,54015,WV
18089,25690,Mingo County,WV,Mingo County,54059,WV


In [37]:
# d4.loc[d4['county_fips'].isna(), 'county_fips'] = df_missing['county_fips'].values

In [38]:
d4.groupby('county_fips')['server_2012'].sum().sum()

5157614.272362255

In [39]:
d5 = d4.groupby('county_fips').agg({
    'server_2012': 'sum',
    'server_2018': 'sum',
    'Server Change': 'sum'}).reset_index()

In [40]:
d4[['server_2012', 'server_2018']].sum(axis=0)

server_2012   5177206.85
server_2018   4261585.89
dtype: float64

In [41]:
d5[['server_2012', 'server_2018']].sum(axis=0)

server_2012   5157614.27
server_2018   4245586.02
dtype: float64

In [42]:
merged_c[['server_2012', 'server_2018']].sum(axis=0)

server_2012   5177206.85
server_2018   4261585.89
dtype: float64

In [43]:
# Write
with pd.ExcelWriter("Datasets/R report.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    d5.to_excel(writer, sheet_name='Server Distribution by fips', index=False)

/opt/miniconda3/envs/env_1/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


PUE and WUE for Small and Midsize DCs in the U.S. 

In [44]:
zip_to_fips = d4.set_index('ZIP CODE')['county_fips'].to_dict()

In [45]:
# 2012 small

a_2012s = df_2012[df_2012['space type'].isin({'small'})].groupby(['space type', 'CENDIV', 'building type'])[['nweight_s']].sum().reset_index()
a_2012s['pba']= a_2012s['building type'].map(pba_id_map)
a_2012s['cdiv num'] = a_2012s['CENDIV']
a_2012s = a_2012s.drop(columns=['CENDIV','building type'])

b_2012s = pd.read_csv("Datasets/employment_by_zip_and_pba_2012.csv", index_col=0)

b_2012s = b_2012s.merge(a_2012s, on=['pba', 'cdiv num'], how='left').fillna(0)
b_2012s['server'] = b_2012s.frac * b_2012s['nweight_s']

b_2012s['ZIP CODE'] = b_2012s['ZIP CODE'].astype(str).str.zfill(5)
b_2012s['county_fips'] = b_2012s['ZIP CODE'].map(zip_to_fips)

c_2012s = b_2012s.groupby(['county_fips', 'pba', 'cdiv num'])[['server']].sum().reset_index()

In [46]:
# 2018 small

a_2018s = df_2018[df_2018['space type'].isin({'small'})].groupby(['space type', 'CENDIV', 'building type'])[['nweight_s']].sum().reset_index()
a_2018s['pba']= a_2018s['building type'].map(pba_id_map)
a_2018s['cdiv num'] = a_2018s['CENDIV']
a_2018s = a_2018s.drop(columns=['CENDIV','building type'])

b_2018s = pd.read_csv("Datasets/employment_by_zip_and_pba_2018.csv", index_col=0)

b_2018s = b_2018s.merge(a_2018s, on=['pba', 'cdiv num'], how='left').fillna(0)
b_2018s['server'] = b_2018s.frac * b_2018s['nweight_s']


b_2018s['ZIP CODE'] = b_2018s['ZIP CODE'].astype(str).str.zfill(5)
b_2018s['county_fips'] = b_2018s['ZIP CODE'].map(zip_to_fips)

c_2018s = b_2018s.groupby(['county_fips', 'pba', 'cdiv num'])[['server']].sum().reset_index()

In [47]:
# 2012 midsize

a_2012m = df_2012[df_2012['space type'].isin({'midsize'})].groupby(['space type', 'CENDIV', 'building type'])[['nweight_s']].sum().reset_index()
a_2012m['pba']= a_2012m['building type'].map(pba_id_map)
a_2012m['cdiv num'] = a_2012m['CENDIV']
a_2012m = a_2012m.drop(columns=['CENDIV','building type'])

b_2012m = pd.read_csv("Datasets/employment_by_zip_and_pba_2012.csv", index_col=0)

b_2012m = b_2012m.merge(a_2012m, on=['pba', 'cdiv num'], how='left').fillna(0)
b_2012m['server'] = b_2012m.frac * b_2012m['nweight_s']

b_2012m['ZIP CODE'] = b_2012m['ZIP CODE'].astype(str).str.zfill(5)
b_2012m['county_fips'] = b_2012m['ZIP CODE'].map(zip_to_fips)

c_2012m = b_2012m.groupby(['county_fips', 'pba', 'cdiv num'])[['server']].sum().reset_index()

In [48]:
# 2018 midsize

a_2018m = df_2018[df_2018['space type'].isin({'midsize'})].groupby(['space type', 'CENDIV', 'building type'])[['nweight_s']].sum().reset_index()
a_2018m['pba']= a_2018m['building type'].map(pba_id_map)
a_2018m['cdiv num'] = a_2018m['CENDIV']
a_2018m = a_2018m.drop(columns=['CENDIV','building type'])

b_2018m = pd.read_csv("Datasets/employment_by_zip_and_pba_2018.csv", index_col=0)

b_2018m = b_2018m.merge(a_2018m, on=['pba', 'cdiv num'], how='left').fillna(0)
b_2018m['server'] = b_2018m.frac * b_2018m['nweight_s']

b_2018m['ZIP CODE'] = b_2018m['ZIP CODE'].astype(str).str.zfill(5)
b_2018m['county_fips'] = b_2018m['ZIP CODE'].map(zip_to_fips)

c_2018m = b_2018m.groupby(['county_fips', 'pba', 'cdiv num'])[['server']].sum().reset_index()

In [49]:
# res_sdiv
# res_mdiv

In [50]:
UEs = pd.read_csv("Datasets/UE_by_CZ_CS.csv", index_col=0)

import ast
UEs['PUE_quantiles'] = UEs['PUE_quantiles'].apply(lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else x)
UEs['WUE_quantiles'] = UEs['WUE_quantiles'].apply(lambda x: np.array(ast.literal_eval(x)) if isinstance(x, str) else x)

In [51]:
# Define a helper function to compute the element-wise mean of quantile lists
def quantile_mean(series):
    arr = np.stack(series.to_numpy())
    return np.mean(arr, axis=0)

# Function to construct new quantiles for 'Air Economizer + Evaporative Cooling (Direct Expansion)'
def construct_new_quantiles(quantile_list):
    new_quantiles = [
        quantile_list[2],  # 3rd quantile -> New 1st quantile (lower bound)
        (quantile_list[2] + quantile_list[3]) * 0.5,  # New 2nd quantile (interpretation)
        quantile_list[3],  # 4th quantile -> New 3rd quantile (median)
        (quantile_list[3] + quantile_list[4]) * 0.5,  # New 4th quantile (interpretation)
        quantile_list[4],  # 5th quantile -> New 5th quantile (upper bound)
    ]
    return new_quantiles

def safe_weighted_average(x, weights):
    weights_sum = np.sum(weights)
    if weights_sum == 0:
        return np.nan
    return np.average(np.stack(x), axis=0, weights=weights).tolist()

In [52]:
## Small DCs
# 'Direct Expansion': 10, "Direct expansion system"
a = UEs[UEs.Case.isin([10])][['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Direct Expansion')
# 'Air Economizer (Direct Expansion)': 12, "Airside economizer (direct expansion system)"
b = UEs[UEs.Case.isin([12])][['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Air Economizer (Direct Expansion)')
# 'Other Chillers': 8, "Water-cooled chiller"; 9, "Air-cooled chiller"
c = UEs[UEs.Case.isin([8, 9])].groupby(['ClimateZone']).agg(
    PUE_quantiles=('PUE_quantiles', quantile_mean),
    WUE_quantiles=('WUE_quantiles', quantile_mean)
).reset_index()[['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Other Chillers')
# 'Direct Expansion + Other Chillers': --- average of 8, 9, 10
d = UEs[UEs.Case.isin([8, 9, 10])].groupby(['ClimateZone']).agg(
    PUE_quantiles=('PUE_quantiles', quantile_mean),
    WUE_quantiles=('WUE_quantiles', quantile_mean)
).reset_index()[['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Direct Expansion + Other Chillers')
# 'Others': --- average of all 8, 9, 10, 12: "Airside economizer (direct expansion system)"
e = UEs[UEs.Case.isin([8, 9, 10, 12])].groupby(['ClimateZone']).agg(
    PUE_quantiles=('PUE_quantiles', quantile_mean),
    WUE_quantiles=('WUE_quantiles', quantile_mean)
).reset_index()[['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Others')

UE_small = pd.concat([a,b,c,d,e], axis=0)

df_UE_small = pd.merge(res_sdiv, UE_small, left_on='Cooling System', right_on='CS', how='inner')

df_UE_small = df_UE_small.groupby(['Year', 'ClimateZone', 'Census Division', 'Div']).agg(
    PUE_quantiles=("PUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=df_UE_small.loc[x.index, 'Percentage']).tolist()),
    WUE_quantiles=("WUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=df_UE_small.loc[x.index, 'Percentage']).tolist())
).reset_index()

In [53]:
df_UE_small

,Year,ClimateZone,Census Division,Div,PUE_quantiles,WUE_quantiles
0,2012,1A,East North Central,3,"[1.9718569339582488, 2.049654003893819, 2.1304...","[0.16213060499458593, 0.19109831184355217, 0.2..."
1,2012,1A,East South Central,6,"[1.9621839265998489, 2.036575023999291, 2.1231...","[0.4056732296052592, 0.45813028524673105, 0.50..."
2,2012,1A,Middle Atlantic,2,"[1.967173359567415, 2.0449008402605187, 2.1267...","[0.20776966067083172, 0.2410172926818865, 0.27..."
3,2012,1A,Mountain,8,"[1.9581172353175433, 2.0346968504385474, 2.119...","[0.34238708383313093, 0.3884608777886466, 0.42..."
4,2012,1A,New England,1,"[1.9751064820554498, 2.050777838392952, 2.1331...","[0.22994981559437627, 0.2657109405570809, 0.29..."
...,...,...,...,...,...,...
265,2018,8,New England,1,"[1.6998806080487654, 1.7805510493289196, 1.849...","[0.1966649705692512, 0.2250031288231857, 0.259..."
266,2018,8,Pacific,9,"[1.6630950407297838, 1.7421095320747333, 1.808...","[0.29752528078000096, 0.3319944391026787, 0.37..."
267,2018,8,South Atlantic,5,"[1.7009212261150806, 1.781281425266431, 1.8503...","[0.22508042603707956, 0.25551978551281573, 0.2..."
268,2018,8,West North Central,4,"[1.6894608395552793, 1.7698996632970263, 1.838...","[0.2044425274653571, 0.23300483240819195, 0.26..."


In [54]:
## Midsize DCs
# 'Other Chillers': --- average of 5, "Water-cooled chiller"; 7, "Air-cooled chiller"
a = UEs[UEs.Case.isin([5, 7])].groupby(['ClimateZone']).agg(
    PUE_quantiles=('PUE_quantiles', quantile_mean),
    WUE_quantiles=('WUE_quantiles', quantile_mean)
).reset_index()[['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Other Chillers')
# 'Air Economizer (Other Chillers)': --- average of 3, "Airside economizer + (water-cooled chiller)"; 6, "Airside economizer + (air-cooled chiller)"; 13, "Airside economizer + (direct expansion system)"
b = UEs[UEs.Case.isin([3, 6, 13])].groupby(['ClimateZone']).agg(
    PUE_quantiles=('PUE_quantiles', quantile_mean),
    WUE_quantiles=('WUE_quantiles', quantile_mean)
).reset_index()[['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Air Economizer (Other Chillers)')
# 'Water Chiller': 5, "Water-cooled chiller"
c = UEs[UEs.Case.isin([5])][['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Water Chiller')
# 'Air Economizer (Water Chiller)':  3, "Airside economizer& (water-cooled chiller)"
d = UEs[UEs.Case.isin([3])][['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Air Economizer (Water Chiller)')
# 'Air Economizer (Air Chiller)': 6, "Airside economizer& (air-cooled chiller)"
e = UEs[UEs.Case.isin([6])][['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Air Economizer (Air Chiller)')
# 'Direct Expansion':  11, "Direct expansion system"
f = UEs[UEs.Case.isin([11])][['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Direct Expansion')
# 'Air Economizer (Direct Expansion)': 13, "Airside economizer (direct expansion system)"
g = UEs[UEs.Case.isin([13])][['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Air Economizer (Direct Expansion)')
# 'Air Economizer + Evaporative Cooling (Direct Expansion)': # upper bound of 0, "Airside economizer + adiabatic cooling + (air-cooled chiller)"; 1, "Airside economizer + adiabatic cooling + (water-cooled chiller)"
h = UEs[UEs.Case.isin([0, 1])].groupby(['ClimateZone']).agg(
    PUE_quantiles=('PUE_quantiles', quantile_mean),
    WUE_quantiles=('WUE_quantiles', quantile_mean)
).reset_index()[['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Air Economizer + Evaporative Cooling (Direct Expansion)')
h["PUE_quantiles"] = h["PUE_quantiles"].apply(construct_new_quantiles)
h["WUE_quantiles"] = h["WUE_quantiles"].apply(construct_new_quantiles)
# 'Air Economizer (Direct Expansion + Other Chillers)':  --- average of 3, 6, 13
i = UEs[UEs.Case.isin([3, 6, 13])].groupby(['ClimateZone']).agg(
    PUE_quantiles=('PUE_quantiles', quantile_mean),
    WUE_quantiles=('WUE_quantiles', quantile_mean)
).reset_index()[['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Air Economizer (Direct Expansion + Other Chillers)')
# 'Air Economizer (Direct Expansion + Water Chiller)': --- average of 3, 13
j = UEs[UEs.Case.isin([3, 13])].groupby(['ClimateZone']).agg(
    PUE_quantiles=('PUE_quantiles', quantile_mean),
    WUE_quantiles=('WUE_quantiles', quantile_mean)
).reset_index()[['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Air Economizer (Direct Expansion + Water Chiller)')
# 'Others': --- average of 3;4 "Waterside economizer + (water-cooled chiller)";5;6;7;11;13;17 "Dry cooler (air-cooled chiller)"
k = UEs[UEs.Case.isin([3,4,5,6,7,11,13,17])].groupby(['ClimateZone']).agg(
    PUE_quantiles=('PUE_quantiles', quantile_mean),
    WUE_quantiles=('WUE_quantiles', quantile_mean)
).reset_index()[['ClimateZone', 'PUE_quantiles', 'WUE_quantiles']].assign(CS = 'Others')

UE_midsize = pd.concat([a,b,c,d,e,f,g,h,i,j,k], axis=0)

df_UE_midsize = pd.merge(res_mdiv, UE_midsize, left_on='Cooling System', right_on='CS', how='inner')

df_UE_midsize = df_UE_midsize.groupby(['Year', 'ClimateZone', 'Census Division', 'Div']).agg(
    PUE_quantiles=("PUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=df_UE_midsize.loc[x.index, 'Percentage']).tolist()),
    WUE_quantiles=("WUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=df_UE_midsize.loc[x.index, 'Percentage']).tolist())
).reset_index()

In [55]:
cz = pd.read_csv("./Datasets/climate_zones.csv", dtype={"State FIPS": str, "County FIPS": str}).assign(
    county_fips=lambda df: df["State FIPS"].str.zfill(2) + df["County FIPS"].str.zfill(3),
    zone=lambda df: df["IECC Climate Zone"].where(
        df["IECC Moisture Regime"].isna() | (df["IECC Moisture Regime"] == "N/A"),
        df["IECC Climate Zone"].astype(str) + df["IECC Moisture Regime"].astype(str)
    )
)

PWUE (2012 small)

In [56]:
c_2012s_cz = pd.merge(c_2012s, cz, on='county_fips', how='left')

c_2012s_cz = pd.merge(
    c_2012s_cz,
    df_UE_small[df_UE_small.Year == 2012],
    left_on=['zone', 'cdiv num'],
    right_on=['ClimateZone', 'Div'],
    how='left'
)

In [57]:
x_small_2012 = pd.DataFrame({col: [np.average(np.stack(c_2012s_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"])[col]), axis=0, 
                               weights=c_2012s_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"])["server"]).tolist()] for col in ["PUE_quantiles", "WUE_quantiles"]})

x_small_2012

,PUE_quantiles,WUE_quantiles
0,"[1.8230347938335005, 1.9135580120486813, 2.000...","[0.2639666513430309, 0.3001837655384829, 0.336..."


In [58]:
quantile_labels = ["5th", "25th", "50th", "75th", "95th"]

df_xs_2012 = []

for i, value in enumerate(x_small_2012["PUE_quantiles"][0]):
    df_xs_2012.append({"value": value, "UE": "PUE", "quantiles": quantile_labels[i], "size": 'small', "year": 2012})

for i, value in enumerate(x_small_2012["WUE_quantiles"][0]):
    df_xs_2012.append({"value": value, "UE": "WUE", "quantiles": quantile_labels[i], "size": 'small', "year": 2012})

df_xs_2012 = pd.DataFrame(df_xs_2012)

df_xs_2012

,value,UE,quantiles,size,year
0,1.82,PUE,5th,small,2012
1,1.91,PUE,25th,small,2012
2,2.00,PUE,50th,small,2012
3,2.13,PUE,75th,small,2012
4,2.29,PUE,95th,small,2012
5,0.26,WUE,5th,small,2012
6,0.30,WUE,25th,small,2012
7,0.34,WUE,50th,small,2012
8,0.37,WUE,75th,small,2012
9,0.42,WUE,95th,small,2012


In [59]:
c_2012s_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"]).groupby(['Census Division']).agg(
    PUE_quantiles=("PUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2012s_cz.loc[x.index, 'server']).tolist()),
    WUE_quantiles=("WUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2012s_cz.loc[x.index, 'server']).tolist())
).reset_index()

,Census Division,PUE_quantiles,WUE_quantiles
0,East North Central,"[1.7911753213955148, 1.8777602499281945, 1.958...","[0.14373746917560967, 0.16876023237598572, 0.1..."
1,East South Central,"[1.8420818478991638, 1.930685860628091, 2.0169...","[0.36979120501603135, 0.4157073536673333, 0.46..."
2,Middle Atlantic,"[1.799454106621956, 1.8869813536219853, 1.9693...","[0.18517459013136406, 0.21392231456920172, 0.2..."
3,Mountain,"[1.818522205142162, 1.914707688674498, 2.00032...","[0.30317242781147286, 0.34275340770415075, 0.3..."
4,New England,"[1.8021082797989503, 1.887147933251395, 1.9683...","[0.2036676370632401, 0.23404168285295884, 0.26..."
5,Pacific,"[1.8026929420417817, 1.8985205547574684, 1.991...","[0.23078557082633927, 0.26399589890849406, 0.2..."
6,South Atlantic,"[1.8403903072366425, 1.9290398408627283, 2.015...","[0.44007741575700343, 0.4920979658021961, 0.54..."
7,West North Central,"[1.799093607438526, 1.8859680303966742, 1.9673...","[0.2017348526982554, 0.23216487240258007, 0.26..."
8,West South Central,"[1.8893936963628912, 1.9833457732737512, 2.075...","[0.18589063750453425, 0.21612585575881454, 0.2..."


In [60]:
c_2012s_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"]).groupby(['pba']).agg(
    PUE_quantiles=("PUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2012s_cz.loc[x.index, 'server']).tolist()),
    WUE_quantiles=("WUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2012s_cz.loc[x.index, 'server']).tolist())
).reset_index()

,pba,PUE_quantiles,WUE_quantiles
0,1,"[1.8093417243545407, 1.9020467547542395, 1.991...","[0.25968559297779936, 0.2954166205290898, 0.33..."
1,2,"[1.822336145935659, 1.9129924218938228, 1.9998...","[0.26499260262125446, 0.30130460272352555, 0.3..."
2,4,"[1.7998629291786745, 1.888963943030045, 1.9736...","[0.1962972957034936, 0.22615890470507646, 0.25..."
3,5,"[1.8200957323731397, 1.9119933962903817, 2.000...","[0.2614090837744035, 0.2973762435062773, 0.333..."
4,6,"[1.8263473825021674, 1.9171609040767614, 2.003...","[0.2670599365461324, 0.30355841682143, 0.34058..."
5,7,"[1.8494912252256972, 1.9408543899599024, 2.029...","[0.23746546200116947, 0.27170091925150885, 0.3..."
6,8,"[1.8201169135344948, 1.910501698082601, 1.9964...","[0.2551070240780265, 0.2904852060737363, 0.326..."
7,11,"[1.8156872047239716, 1.9055575915912282, 1.991...","[0.31104501408111357, 0.35126508249872157, 0.3..."
8,12,"[1.8268258388792173, 1.9174945625257898, 2.004...","[0.2594925840553624, 0.29539602932432013, 0.33..."
9,13,"[1.8223873404496715, 1.9124673085846675, 1.998...","[0.24943871761731895, 0.28435192488441696, 0.3..."


PWUE (2018 small)

In [61]:
c_2018s_cz = pd.merge(c_2018s, cz, on='county_fips', how='left')

c_2018s_cz = pd.merge(
    c_2018s_cz,
    df_UE_small[df_UE_small.Year == 2018],
    left_on=['zone', 'cdiv num'],
    right_on=['ClimateZone', 'Div'],
    how='left'
)

In [62]:
x_small_2018 = pd.DataFrame({col: [np.average(np.stack(c_2018s_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"])[col]), axis=0, 
                               weights=c_2018s_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"])["server"]).tolist()] for col in ["PUE_quantiles", "WUE_quantiles"]})
x_small_2018

,PUE_quantiles,WUE_quantiles
0,"[1.817980846367883, 1.9089379534304707, 1.9950...","[0.21023620493749812, 0.24151392806001865, 0.2..."


In [63]:
quantile_labels = ["5th", "25th", "50th", "75th", "95th"]

df_xs_2018 = []

for i, value in enumerate(x_small_2018["PUE_quantiles"][0]):
    df_xs_2018.append({"value": value, "UE": "PUE", "quantiles": quantile_labels[i], "size": 'small', "year": 2018})

for i, value in enumerate(x_small_2018["WUE_quantiles"][0]):
    df_xs_2018.append({"value": value, "UE": "WUE", "quantiles": quantile_labels[i], "size": 'small', "year": 2018})

df_xs_2018 = pd.DataFrame(df_xs_2018)

df_xs_2018

,value,UE,quantiles,size,year
0,1.82,PUE,5th,small,2018
1,1.91,PUE,25th,small,2018
2,2.00,PUE,50th,small,2018
3,2.13,PUE,75th,small,2018
4,2.28,PUE,95th,small,2018
5,0.21,WUE,5th,small,2018
6,0.24,WUE,25th,small,2018
7,0.27,WUE,50th,small,2018
8,0.31,WUE,75th,small,2018
9,0.35,WUE,95th,small,2018


In [64]:
c_2018s_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"]).groupby(['Census Division']).agg(
    PUE_quantiles=("PUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2018s_cz.loc[x.index, 'server']).tolist()),
    WUE_quantiles=("WUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2018s_cz.loc[x.index, 'server']).tolist())
).reset_index()

,Census Division,PUE_quantiles,WUE_quantiles
0,East North Central,"[1.7947573154915042, 1.8810722213999846, 1.962...","[0.14901038551447168, 0.17456292982028337, 0.2..."
1,East South Central,"[1.85654352296667, 1.9466205666604697, 2.03459...","[0.235946038922167, 0.27036946063935174, 0.306..."
2,Middle Atlantic,"[1.8009977446514795, 1.8877162912060617, 1.969...","[0.24513356683305182, 0.2791517750321344, 0.31..."
3,Mountain,"[1.8427361707650303, 1.9406444094166906, 2.028...","[0.14477774129563398, 0.17104333539911867, 0.1..."
4,New England,"[1.7925456654533223, 1.8779452021681164, 1.958...","[0.20568170900171465, 0.23602156499451415, 0.2..."
5,Pacific,"[1.7367646553797775, 1.831907204547718, 1.9211...","[0.3151726828378842, 0.3543190154429726, 0.391..."
6,South Atlantic,"[1.8605438081747263, 1.9507889527661526, 2.038...","[0.2468011907121755, 0.28204806412155026, 0.31..."
7,West North Central,"[1.786502114499526, 1.8737276579230362, 1.9541...","[0.21469708793809414, 0.24588028237995146, 0.2..."
8,West South Central,"[1.8800198355796638, 1.9754385750752779, 2.067...","[0.1491830323305126, 0.17583298792501148, 0.20..."


In [65]:
c_2018s_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"]).groupby(['pba']).agg(
    PUE_quantiles=("PUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2018s_cz.loc[x.index, 'server']).tolist()),
    WUE_quantiles=("WUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2018s_cz.loc[x.index, 'server']).tolist())
).reset_index()

,pba,PUE_quantiles,WUE_quantiles
0,1,"[1.8132286248217242, 1.9053125302473002, 1.993...","[0.2665918951257511, 0.3027815226553784, 0.338..."
1,2,"[1.8174928792038623, 1.9085661911153886, 1.994...","[0.21023271081762643, 0.24149365083337868, 0.2..."
2,4,"[1.802405597213672, 1.889970299410678, 1.97294...","[0.2178998657016403, 0.24951265178135992, 0.28..."
3,5,"[1.8189041807152884, 1.9091854786458187, 1.994...","[0.2088141540295628, 0.2400039225247603, 0.272..."
4,6,"[1.8294663103953626, 1.9201197926682447, 2.006...","[0.1774438264206137, 0.20592500762693983, 0.23..."
5,7,"[1.8164829169045238, 1.9069995659752637, 1.992...","[0.20458334742039838, 0.23529650840439645, 0.2..."
6,8,"[1.8232083623574222, 1.91480216351055, 2.00121...","[0.21237575374635592, 0.24397975682974762, 0.2..."
7,11,"[1.7768098094417697, 1.8681916525807567, 1.954...","[0.2623355932458901, 0.2974819640118037, 0.332..."
8,12,"[1.8256541555046983, 1.9167175670278125, 2.003...","[0.21891294904026184, 0.25106680759616246, 0.2..."
9,13,"[1.812342118954202, 1.9026705478451196, 1.9882...","[0.20938487273800577, 0.24050867791241246, 0.2..."


PWUE (2012 midsize)

In [66]:
c_2012m_cz = pd.merge(c_2012m, cz, on='county_fips', how='left')

c_2012m_cz = pd.merge(
    c_2012m_cz,
    df_UE_midsize[df_UE_midsize.Year == 2012],
    left_on=['zone', 'cdiv num'],
    right_on=['ClimateZone', 'Div'],
    how='left'
)

In [67]:
x_mid_2012 = pd.DataFrame({col: [np.average(np.stack(c_2012m_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"])[col]), axis=0, 
                               weights=c_2012m_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"])["server"]).tolist()] for col in ["PUE_quantiles", "WUE_quantiles"]})
x_mid_2012

,PUE_quantiles,WUE_quantiles
0,"[1.4595518167266748, 1.5415770406345397, 1.613...","[0.5152597273985023, 0.5981562898113177, 0.671..."


In [68]:
quantile_labels = ["5th", "25th", "50th", "75th", "95th"]

df_xm_2012 = []

for i, value in enumerate(x_mid_2012["PUE_quantiles"][0]):
    df_xm_2012.append({"value": value, "UE": "PUE", "quantiles": quantile_labels[i], "size": 'midsize', "year": 2012})

for i, value in enumerate(x_mid_2012["WUE_quantiles"][0]):
    df_xm_2012.append({"value": value, "UE": "WUE", "quantiles": quantile_labels[i], "size": 'midsize', "year": 2012})

df_xm_2012 = pd.DataFrame(df_xm_2012)

df_xm_2012

,value,UE,quantiles,size,year
0,1.46,PUE,5th,midsize,2012
1,1.54,PUE,25th,midsize,2012
2,1.61,PUE,50th,midsize,2012
3,1.70,PUE,75th,midsize,2012
4,1.83,PUE,95th,midsize,2012
5,0.52,WUE,5th,midsize,2012
6,0.60,WUE,25th,midsize,2012
7,0.67,WUE,50th,midsize,2012
8,0.75,WUE,75th,midsize,2012
9,0.86,WUE,95th,midsize,2012


In [69]:
c_2012m_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"]).groupby(['Census Division']).agg(
    PUE_quantiles=("PUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2012m_cz.loc[x.index, 'server']).tolist()),
    WUE_quantiles=("WUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2012m_cz.loc[x.index, 'server']).tolist())
).reset_index()

,Census Division,PUE_quantiles,WUE_quantiles
0,East North Central,"[1.4287383825094504, 1.5081596321913249, 1.577...","[0.34486751039725677, 0.39926932582987307, 0.4..."
1,East South Central,"[1.597624633026102, 1.6982280701861932, 1.7852...","[0.255936785137587, 0.309774688997677, 0.35791..."
2,Middle Atlantic,"[1.4174220933491213, 1.4951289221984336, 1.562...","[0.4313048970740757, 0.4935295800432308, 0.548..."
3,Mountain,"[1.5618461454110197, 1.6527588486800588, 1.736...","[0.4322591751891807, 0.49965513521415567, 0.56..."
4,New England,"[1.4150120967698698, 1.4916178490988172, 1.558...","[0.3792538638534413, 0.43074761734478273, 0.47..."
5,Pacific,"[1.3850130696061087, 1.4619763644752317, 1.529...","[0.5875010893056463, 0.6991631708237014, 0.803..."
6,South Atlantic,"[1.4933172514069974, 1.5761046424203342, 1.649...","[0.7378231155667888, 0.8434532908719296, 0.933..."
7,West North Central,"[1.3873597316947885, 1.4593937505648715, 1.522...","[0.4127893295645132, 0.5073315847209728, 0.583..."
8,West South Central,"[1.5068664305514388, 1.5914085212788223, 1.666...","[0.7625287557669377, 0.8698468731998218, 0.961..."


In [70]:
c_2012m_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"]).groupby('pba').agg(
    PUE_quantiles=("PUE_quantiles", lambda x: safe_weighted_average(x, c_2012m_cz.loc[x.index, 'server'])),
    WUE_quantiles=("WUE_quantiles", lambda x: safe_weighted_average(x, c_2012m_cz.loc[x.index, 'server']))
).reset_index()

,pba,PUE_quantiles,WUE_quantiles
0,1,NaN,NaN
1,2,"[1.4589713435639873, 1.5408484435151983, 1.612...","[0.5161239383949108, 0.5974623669663293, 0.669..."
2,4,"[1.532149240318989, 1.6205777480854033, 1.7013...","[0.4495451895982308, 0.5218545570979313, 0.589..."
3,5,"[1.3904787525182845, 1.4680191662266078, 1.535...","[0.5756892207742963, 0.6849517458754905, 0.785..."
4,6,NaN,NaN
5,7,"[1.43379968493085, 1.5129257223727783, 1.58207...","[0.4799086946167205, 0.5503919481392673, 0.612..."
6,8,"[1.4170797050553345, 1.4931722678788057, 1.560...","[0.4726207756853826, 0.5612495570712948, 0.635..."
7,11,"[1.4141324174117484, 1.4912842265061943, 1.558...","[0.4298079093919483, 0.4920584573250235, 0.547..."
8,12,NaN,NaN
9,13,"[1.4972552531396288, 1.581246952197925, 1.6558...","[0.7153724440986882, 0.8176015074054281, 0.905..."


PWUE (2018 midsize)

In [71]:
c_2018m_cz = pd.merge(c_2018m, cz, on='county_fips', how='left')

c_2018m_cz = pd.merge(
    c_2018m_cz,
    df_UE_midsize[df_UE_midsize.Year == 2018],
    left_on=['zone', 'cdiv num'],
    right_on=['ClimateZone', 'Div'],
    how='left'
)

In [72]:
x_mid_2018 = pd.DataFrame({col: [np.average(np.stack(c_2018m_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"])[col]), axis=0, 
                               weights=c_2018m_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"])["server"]).tolist()] for col in ["PUE_quantiles", "WUE_quantiles"]})
x_mid_2018

,PUE_quantiles,WUE_quantiles
0,"[1.4259330286666119, 1.5031290869279674, 1.571...","[0.5583319978699492, 0.6577412657128422, 0.741..."


In [73]:
quantile_labels = ["5th", "25th", "50th", "75th", "95th"]

df_xm_2018 = []

for i, value in enumerate(x_mid_2018["PUE_quantiles"][0]):
    df_xm_2018.append({"value": value, "UE": "PUE", "quantiles": quantile_labels[i], "size": 'midsize', "year": 2018})

for i, value in enumerate(x_mid_2018["WUE_quantiles"][0]):
    df_xm_2018.append({"value": value, "UE": "WUE", "quantiles": quantile_labels[i], "size": 'midsize', "year": 2018})

df_xm_2018 = pd.DataFrame(df_xm_2018)

df_xm_2018 

,value,UE,quantiles,size,year
0,1.43,PUE,5th,midsize,2018
1,1.50,PUE,25th,midsize,2018
2,1.57,PUE,50th,midsize,2018
3,1.65,PUE,75th,midsize,2018
4,1.76,PUE,95th,midsize,2018
5,0.56,WUE,5th,midsize,2018
6,0.66,WUE,25th,midsize,2018
7,0.74,WUE,50th,midsize,2018
8,0.83,WUE,75th,midsize,2018
9,0.96,WUE,95th,midsize,2018


In [74]:
c_2018m_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"]).groupby(['Census Division']).agg(
    PUE_quantiles=("PUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2018m_cz.loc[x.index, 'server']).tolist()),
    WUE_quantiles=("WUE_quantiles", lambda x: np.average(np.stack(x), axis=0, weights=c_2018m_cz.loc[x.index, 'server']).tolist())
).reset_index()

,Census Division,PUE_quantiles,WUE_quantiles
0,East North Central,"[1.384681468853951, 1.4569988485948782, 1.5214...","[0.4123776799406195, 0.5072652890679747, 0.585..."
1,East South Central,"[1.4428667675570233, 1.523977684663625, 1.5923...","[0.20806960120588547, 0.2530830349014603, 0.28..."
2,Middle Atlantic,"[1.3958889973884134, 1.471404293987699, 1.5371...","[0.3828802899315442, 0.4698279482217563, 0.539..."
3,Mountain,"[1.3964080846225522, 1.47817968257096, 1.55027...","[0.2333649891161906, 0.31179376684993937, 0.37..."
4,New England,"[1.4022906857463442, 1.4753017332737175, 1.541...","[0.5715028290009881, 0.6636271633094575, 0.744..."
5,Pacific,"[1.4454367277451776, 1.5279096580065925, 1.597...","[0.2476113413091088, 0.29544384793307693, 0.34..."
6,South Atlantic,"[1.4540320651399148, 1.5313783029716332, 1.601...","[0.9032150789286211, 1.0517515090211271, 1.172..."
7,West North Central,"[1.4022100842850518, 1.4657921950231598, 1.525...","[1.681134011087289, 1.8373398029464694, 1.9846..."
8,West South Central,"[1.4603172740856383, 1.5390737162189851, 1.610...","[0.9904653539150174, 1.1547394859357198, 1.287..."


In [75]:
c_2018m_cz.dropna(subset=["PUE_quantiles", "WUE_quantiles"]).groupby('pba').agg(
    PUE_quantiles=("PUE_quantiles", lambda x: safe_weighted_average(x, c_2018m_cz.loc[x.index, 'server'])),
    WUE_quantiles=("WUE_quantiles", lambda x: safe_weighted_average(x, c_2018m_cz.loc[x.index, 'server']))
).reset_index()

,pba,PUE_quantiles,WUE_quantiles
0,1,NaN,NaN
1,2,"[1.420313023637679, 1.4967952138967764, 1.5642...","[0.5646254601606253, 0.6650377673928012, 0.749..."
2,4,"[1.4427384278215398, 1.5247892091996873, 1.593...","[0.24360762840044647, 0.29058650749036463, 0.3..."
3,5,"[1.4188799033526065, 1.4946261791040425, 1.560...","[0.5750736420265606, 0.6586228191305066, 0.735..."
4,6,NaN,NaN
5,7,"[1.4335432694172925, 1.512820533898871, 1.5805...","[0.2665792328499678, 0.32277311588195357, 0.36..."
6,8,"[1.4427736845659127, 1.5230093574923915, 1.592...","[0.4755697848027429, 0.5620405598058926, 0.637..."
7,11,NaN,NaN
8,12,NaN,NaN
9,13,"[1.4428721196264074, 1.5251707083204145, 1.594...","[0.2581387555120314, 0.3092290344780456, 0.359..."


In [76]:
d6 = pd.concat([df_xs_2012, df_xs_2018, df_xm_2012, df_xm_2018])

# Write
with pd.ExcelWriter("Datasets/R report.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    d6.to_excel(writer, sheet_name='UE by DC size and year', index=False)

/opt/miniconda3/envs/env_1/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [77]:
df_xm_2012

,value,UE,quantiles,size,year
0,1.46,PUE,5th,midsize,2012
1,1.54,PUE,25th,midsize,2012
2,1.61,PUE,50th,midsize,2012
3,1.70,PUE,75th,midsize,2012
4,1.83,PUE,95th,midsize,2012
5,0.52,WUE,5th,midsize,2012
6,0.60,WUE,25th,midsize,2012
7,0.67,WUE,50th,midsize,2012
8,0.75,WUE,75th,midsize,2012
9,0.86,WUE,95th,midsize,2012


In [78]:
df_xm_2018

,value,UE,quantiles,size,year
0,1.43,PUE,5th,midsize,2018
1,1.50,PUE,25th,midsize,2018
2,1.57,PUE,50th,midsize,2018
3,1.65,PUE,75th,midsize,2018
4,1.76,PUE,95th,midsize,2018
5,0.56,WUE,5th,midsize,2018
6,0.66,WUE,25th,midsize,2018
7,0.74,WUE,50th,midsize,2018
8,0.83,WUE,75th,midsize,2018
9,0.96,WUE,95th,midsize,2018
